# Condensé de la séance précédente

In [1]:
# imports
from requests import get
import pandas as pd
import json
import re

In [3]:
# TOUTES les constantes globales
ADRESSE = "https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/brute.json"
NEUF = {'Appartement neuf', 'Maison / Villa neuve'}
MAISON =  {'Maison / Villa', 'Maison / Villa neuve'}
GENRES_VALIDES = {'Appartement', 'Appartement neuf', 'Maison / Villa', 'Maison / Villa neuve'}
MOTIF_SURFACE = re.compile("^.*?([0-9]+(,[0-9]+)?) m².*$")

**REMARQUE** on va documenter avec des commentaires.
De fait, on se rend compte qu'il serait préférable de transformer chaque étape en une fonction qui prendrait comme nom le commentaire.

In [5]:
# génération du dataframe
page = get(ADRESSE)
contenu = page.text
data = [json.loads(ligne) for ligne in contenu.splitlines()]
df = pd.DataFrame(data=data)

# gestion id
df.drop_duplicates(subset="id", inplace=True)
df.drop(columns="id", inplace=True)
df.reset_index(inplace=True, drop=True)

# gestion genre
df.drop(df[~df["genre"].isin(GENRES_VALIDES)].index, inplace=True)
df.reset_index(inplace=True, drop=True)
df["neuf"] = df["genre"].isin(NEUF)
df["maison"] = df["genre"].isin(MAISON)
df.drop(columns="genre", inplace=True)
df.reset_index(inplace=True, drop=True)

# gestion prix
df["prix_1"] = df["prix"].str.replace("€", "").str.replace(" ","").str.replace("HH", "")
df.drop(df[df["prix_1"] == ""].index, inplace=True)
df["target_prix"] = df["prix_1"].astype(float)
df.drop(columns="prix_1", inplace=True)
df.drop(columns="prix", inplace=True)
df.reset_index(inplace=True, drop=True)

# gestion pcs
df.drop(df[~df["pcs"].str.match(MOTIF_SURFACE)].index, inplace=True)
df.reset_index(inplace=True, drop=True)
df["surface"] = df["pcs"].str.extract(MOTIF_SURFACE)[0]
df.reset_index(inplace=True, drop=True)
df

,pcs,desc,lien,neuf,maison,target_prix,surface
0,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...,False,False,374400.0,90
1,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...,False,False,499200.0,"146,27"
2,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...,False,False,499200.0,110
3,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...,False,True,508000.0,132
4,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...,False,True,676000.0,185
...,...,...,...,...,...,...,...
1620,"3 p 66,3 m²",Maisons de ville 3 chambres avec jardin ou app...,https://www.selogerneuf.com/annonces/achat/app...,True,False,254900.0,"66,3"
1621,2 p 1 ch 42 m²,"TOURS - Fontaines, Appartement de type 2 compr...",https://www.seloger.com/annonces/achat/apparte...,False,False,61500.0,42
1622,3 p 2 ch 76 m²,TOURS - Appartement de Type 3 comprenant séjou...,https://www.seloger.com/annonces/achat/apparte...,False,False,108500.0,76
1623,"4 p 84,4 m²",Maisons de ville 3 chambres avec jardin ou app...,https://www.selogerneuf.com/annonces/achat/mai...,True,True,320000.0,"84,4"


# Exercice

Finir de gérer la colonne `pcs`.

In [6]:
MOTIF_PIECES = re.compile("^.*?([0-9]+) p.*$")

In [10]:
df["pcs"].str.match(MOTIF_PIECES).unique()

array([ True])

In [12]:
df["pieces"] = df["pcs"].str.extract(MOTIF_PIECES)

In [13]:
MOTIF_CHAMBRES = re.compile("^.*?([0-9]+) ch.*$")

In [17]:
df["pcs"].str.match(MOTIF_CHAMBRES).value_counts()

True     1450
False     175
Name: pcs, dtype: int64

**REMARQUE** on a plusieurs possibilités.

1. Abandonner l'idée d'avoir une variable explicative `chambres` en considérant que `pieces` et `surface` sont suffisantes.
2. On peut faire `drop` sur les lignes de l'échantillon sans information `chambres`, 1450 éléments restant une quantité raisonnable dans l'échantillon.
3. On peut remplir les informations manquantes avec  `NaN`.
On peut ensuite soit remplir le résultat avec une valeur ad hoc, soit utiliser la corrélation avec `pieces` et `surface` pour faire une régression.

In [18]:
df["chambres"] = df["pcs"].str.extract(MOTIF_CHAMBRES)

In [20]:
df.drop(columns="pcs", inplace=True)
df

,desc,lien,neuf,maison,target_prix,surface,pieces,chambres
0,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...,False,False,374400.0,90,3,2
1,TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...,False,False,499200.0,"146,27",5,4
2,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...,False,False,499200.0,110,5,3
3,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...,False,True,508000.0,132,6,4
4,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...,False,True,676000.0,185,7,5
...,...,...,...,...,...,...,...,...
1620,Maisons de ville 3 chambres avec jardin ou app...,https://www.selogerneuf.com/annonces/achat/app...,True,False,254900.0,"66,3",3,NaN
1621,"TOURS - Fontaines, Appartement de type 2 compr...",https://www.seloger.com/annonces/achat/apparte...,False,False,61500.0,42,2,1
1622,TOURS - Appartement de Type 3 comprenant séjou...,https://www.seloger.com/annonces/achat/apparte...,False,False,108500.0,76,3,2
1623,Maisons de ville 3 chambres avec jardin ou app...,https://www.selogerneuf.com/annonces/achat/mai...,True,True,320000.0,"84,4",4,NaN


In [22]:
df.dtypes

desc            object
lien            object
neuf              bool
maison            bool
target_prix    float64
surface         object
pieces          object
chambres        object
dtype: object

# Exercice

Convertissez les trois dernières colonnes en float.

In [27]:
df["chambres"] = df["chambres"].astype(float)
df["pieces"] = df["pieces"].astype(float)

In [28]:
df.dtypes

desc            object
lien            object
neuf              bool
maison            bool
target_prix    float64
surface         object
pieces         float64
chambres       float64
dtype: object

In [31]:
df["surface"] = df["surface"].str.replace(",", ".").astype(float)

In [32]:
df.dtypes

desc            object
lien            object
neuf              bool
maison            bool
target_prix    float64
surface        float64
pieces         float64
chambres       float64
dtype: object

**REMARQUE** à ce stade on a la cible et 5 variables explicatives numériques.
Avant de chercher à exploiter `desc` et `lien` qui sont délicates, on va essayer d'entrainer des modèles sur ce dataframe.

# Exercice

Entrainer des modèles pour estimer le pouvoir explicatif des variables explicatives déjà présentes.

On cherchera à utiliser la fonction `GridSearchCV` de sckikit-learn pour simplifier le code.

On réfléchira aussi à l'imputation sur les valeurs manquantes de la variable `chambres`.
On pourra en particulier utiliser le module `sklearn.impute` pour voir différentes possibilités.

In [37]:
import numpy as np

In [47]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.pipeline import Pipeline, FeatureUnion

## Récupéraration des tableaux numériques

In [42]:
X = df[["neuf", "maison", "pieces", "surface", "chambres"]].astype(float).values
print(X.dtype)
X

float64


array([[  0.  ,   0.  ,   3.  ,  90.  ,   2.  ],
       [  0.  ,   0.  ,   5.  , 146.27,   4.  ],
       [  0.  ,   0.  ,   5.  , 110.  ,   3.  ],
       ...,
       [  0.  ,   0.  ,   3.  ,  76.  ,   2.  ],
       [  1.  ,   1.  ,   4.  ,  84.4 ,    nan],
       [  1.  ,   1.  ,   4.  ,  84.4 ,    nan]])

In [44]:
y = df["target_prix"].values
y

array([374400., 499200., 499200., ..., 108500., 320000., 320000.])

In [46]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)

# Sélection du meilleur modèle

Attention, comme il y a plusieurs stratégie d'imputation, le choix du type d'imputation doit être considéré comme un hyperparamètre.

In [48]:
ri = Pipeline(
    [
        ("imputation", SimpleImputer()),
        ("entrainement", Ridge()),
    ]
)

**REMARQUE** dans `ri` il y a deux hyperparamètres: 

1. `strategy` en provenance du `SimpleImputer`
2. `alpha` en provenance de `Ridge`

In [49]:
ri.get_params()

{'memory': None,
 'steps': [('imputation', SimpleImputer()), ('entrainement', Ridge())],
 'verbose': False,
 'imputation': SimpleImputer(),
 'entrainement': Ridge(),
 'imputation__add_indicator': False,
 'imputation__copy': True,
 'imputation__fill_value': None,
 'imputation__keep_empty_features': False,
 'imputation__missing_values': nan,
 'imputation__strategy': 'mean',
 'imputation__verbose': 'deprecated',
 'entrainement__alpha': 1.0,
 'entrainement__copy_X': True,
 'entrainement__fit_intercept': True,
 'entrainement__max_iter': None,
 'entrainement__positive': False,
 'entrainement__random_state': None,
 'entrainement__solver': 'auto',
 'entrainement__tol': 0.0001}

On voit que dans `ri` les hyperparamètres qu'on veut explorer sont maintenant nommés:
1. `imputation__strategy` 
2. `entrainement__alpha`

On a juste ajouté le nom de l'étape comme préfixe.

In [61]:
gr = GridSearchCV(
    estimator=ri,
    param_grid={
        "imputation__strategy": ["mean", "median", "most_frequent"],
        "entrainement__alpha": [2  ** p for p in range(-6, 7)],
    }
)
gr.fit(X_tr, y_tr)
indice_meilleur = gr.cv_results_["rank_test_score"].argmin()
print(indice_meilleur)
print(gr.cv_results_["params"][indice_meilleur])
print(gr.cv_results_["rank_test_score"][indice_meilleur])
print(gr.cv_results_["mean_test_score"][indice_meilleur])
print(gr.cv_results_["std_test_score"][indice_meilleur])

In [62]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Gérer SimpleImputer et MPLRegressor
mlp  = Pipeline(
    [
        ("imputation", SimpleImputer()),
        ("echelle", MinMaxScaler()),
        ("entrainement", MLPRegressor()),
    ]
)
gr = GridSearchCV(
    estimator=mlp,
    param_grid={
        "imputation__strategy": ["mean", "median", "most_frequent"],
        "entrainement__hidden_layer_sizes": [(50, ), (100,), (150,) ,(200,), (250,)],
        "entrainement__max_iter": [1000],
    }
)
gr.fit(X_tr, y_tr)
indice_meilleur = gr.cv_results_["rank_test_score"].argmin()
print(indice_meilleur)
print(gr.cv_results_["params"][indice_meilleur])
print(gr.cv_results_["rank_test_score"][indice_meilleur])
print(gr.cv_results_["mean_test_score"][indice_meilleur])
print(gr.cv_results_["std_test_score"][indice_meilleur])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\neural_network\_m

# Vérification du surapprentissage

TBD...